In [ ]:
import pandas as pd
import json
import ast
import pyarrow
%load_ext autoreload
%autoreload 2
import funciones_etl

import warnings
warnings.filterwarnings("ignore")

Abrimos el archivo australian_users_items

In [3]:
ruta = "../Datasets/australian_users_items.json"

In [4]:
fila = []
with open(ruta, encoding="utf-8") as f:
    for line in f.readlines():
        fila.append(ast.literal_eval(line))

In [5]:
# Cargamos el archivo en un dataframe
df_items = pd.DataFrame(fila)
df_items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [6]:
df_items.shape

(88310, 5)

Obtenemos un dataframe con 5 columnas pero la columna items esta anidada, el cual procedemos a transformar desanidando y descartando las columnas innecesarias

In [7]:
# Crear una lista para almacenar los registros desglosados
items_rec = []
for index, row in df_items.iterrows():
    user_id = row['user_id'] # Agregar el user_id 
    items_list = row['items'] # Extraer todos los datos de la coumna items de df_users_items 
    for item in items_list:
        item_record = {
            'user_id': user_id,
            'item_id': item['item_id'],
            'item_name': item['item_name'],
            'playtime_forever': item['playtime_forever'],
            'playtime_2weeks': item['playtime_2weeks']
        }
        items_rec.append(item_record)

df_items2 = pd.DataFrame(items_rec)

In [8]:
#Observamos el dataframe
df_items2.head()

,user_id,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,10,Counter-Strike,6,0
1,76561197970982479,20,Team Fortress Classic,0,0
2,76561197970982479,30,Day of Defeat,7,0
3,76561197970982479,40,Deathmatch Classic,0,0
4,76561197970982479,50,Half-Life: Opposing Force,0,0


In [9]:
# eliminamos la columna playtime_2weeks
df_items2 = df_items2.drop("playtime_2weeks", axis=1)

In [10]:
# Verificamos el tipo de datos y los valores nulos 
funciones_etl.verificar_datos(df_items2)

,columna,tipo_dato,datos_nulos,porcentaje_nulos,porcentaje_no_nulos
0,user_id,[<class 'str'>],0,0.0,100.0
1,item_id,[<class 'str'>],0,0.0,100.0
2,item_name,[<class 'str'>],0,0.0,100.0
3,playtime_forever,[<class 'int'>],0,0.0,100.0


In [15]:
# verificamos el la informacion basica
df_items2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 4 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   item_id           object
 2   item_name         object
 3   playtime_forever  int64 
dtypes: int64(1), object(3)
memory usage: 157.3+ MB


In [18]:
# Optimizando el dataframe
df_items2 = df_items2[:1000000]

In [19]:
df_items2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 4 columns):
 #   Column            Non-Null Count    Dtype 
---  ------            --------------    ----- 
 0   user_id           1000000 non-null  object
 1   item_id           1000000 non-null  object
 2   item_name         1000000 non-null  object
 3   playtime_forever  1000000 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 30.5+ MB


Guardamos el dataframe en formato parquet para estar comprimido

In [26]:
df_items2.to_parquet("../Datasets/australian_items_limpio.parquet")